In [11]:
import os
import json
import pandas as pd
import traceback

In [12]:
!pip install langchain-community
!pip install --upgrade langchain
!pip show langchain-community

Name: langchain-community
Version: 0.3.0
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\armst\mcqgen\env\lib\site-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [13]:
from langchain.chat_models import ChatOpenAI

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
KEY=os.getenv("OPENAI_API_KEY")

In [16]:
print(KEY)

sk-30gfZtow6BxC9sbSSrMAFgS0-afdbHxYsUHhVHuryIT3BlbkFJzeWL8U7weTvfaKe_jK1wKyQ1IafMgmSmKqkTYzcn8A
